In [1]:
import os
import sys
import config as cfg
import time

import pandas as pd
import numpy as np

In [2]:
PROJECT_NAME = 'project name'
PROJECT_START_DATE = 'start date' #ENGLISH DATE FORMAT
PROJECT_END_DATE = '' #ENGLISH DATE FORMAT

In [3]:
#ROOT, PROJECT_ROOT, PACKAGE_ROOT = config.configure_environment('Local',PROJECT_NAME)

config = cfg.privateConfig(cfg.OS, PROJECT_NAME)

The haribo package provides the following functions to use in Jupyter:

set_screen_width() - sets cells to be 90% of screen width

show_df(dataFrame, allRows = False) - shows all columns of a dataFrame when displayed

pickle_it(data, path) - safely pickles a file

from_pickle_jar(path) - safely loads a pickled file



In [4]:
from UtilityPackagesV2.dbutility import gnrl_database_interaction as gdbi
from UtilityPackagesV2.dbutility import credentials
from UtilityPackagesV2.apiutility import gnrl_api_interaction as gapi
from CodeCandy.haribo import jupyter as candy

# Create a connection object

In [ ]:
logFileName = PROJECT_NAME + '.log'
_ = config.set_logging(logFileName)

newUser = cfg.credentials()

dataIn = {'credentials': newUser,
         'logFileName': 'LogMIQuery.log'}

connection = gdbi.MIConnection(config,credentials=newUser, logFileName=logFileName)
connection = gdbi.GBQConnection(config, credentials=newUser,logFileName=logFileName)
connection2 = gdbi.GBQConnection(config, credentials=newUser,logFileName=logFileName,useLegacy=False, projectId='on-the-beach---attribution',if_exists='replace') # push to BQ
connection = gapi.AdwordsAPIConnection(config, os.path.join(config.PROJECT_ROOT,'Materials', 'googleads.yaml'), credentials=newUser, logFileName = logFileName)
connection = gapi.GaAPIConnection(config, credentials=newUser, logFileName = logFileName)

# Create a connection object handler

In [ ]:

sqlConnection = gdbi.ExecuteSQL(
    connection=connection, 
    query = os.path.join(config.PROJECT_ROOT,'Materials', 'hourlyReportUK.sql'),
    outputPath = os.path.join(config.PROJECT_ROOT,'Data', 'HourlyReportDataMI_29_10_2018.csv')
)

query = {
    'reportRequests': [
    {
      'viewId': '90231540' ,
      'dateRanges': [{'startDate': '7daysAgo', 'endDate': 'today'}],
      'metrics': [{'expression': 'ga:sessions'}],
      'dimensions': [{'name': 'ga:country'}]
    }]}

sqlConnection = gapi.ExecuteAPICall(
    connection=connection, 
    query=query,
    outputPath=os.path.join(config.PROJECT_ROOT,'Data', 'GA_API_data.csv')
)

# Pull data

In [ ]:
message, data = sqlConnection.execute_script()
data

# Put code here